（20分） 利用surprise库编写一个协同过滤算法，上传调试结果。

In [14]:
from surprise import Dataset, Reader, KNNWithZScore, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
import pandas as pd
import numpy as np

In [25]:
# 读取数据
data=Dataset.load_builtin('ml-100k')

In [33]:
# 定义参数范围
param_grid = {'k': list(np.arange(10, 61, 5)), 
              'min_k': list(range(1, 6)),
              'sim_options': {'name': ['cosine', 'MSD', 'pearson'],
                              'min_support': [1, 3, 5]}}
# 搜索最优参数
gs = GridSearchCV(KNNWithZScore, param_grid, measures=['rmse', 'mae'], n_jobs=-1, cv=3)
# 训练
gs.fit(data)

In [34]:
# 以RMSE评分，显示最优模型参数
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9524553901015985
{'k': 60, 'min_k': 3, 'sim_options': {'name': 'pearson', 'min_support': 5, 'user_based': True}}


In [36]:
# 以MAE评分，显示最优模型参数
print(gs.best_score['mae'])
print(gs.best_params['mae'])

0.743227115534014
{'k': 55, 'min_k': 2, 'sim_options': {'name': 'pearson', 'min_support': 5, 'user_based': True}}


In [37]:
# 以RMSE评分最优的参数建立模型
best_knn = KNNWithZScore(k=60, min_k=3, sim_options={'name':'pearson', 'min_support':5, 'user_base':True})
# 以整个数据训练
best_knn.fit(data.build_full_trainset())

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [44]:
# 预测评分与实际评分对比
best_knn.predict('87', '1016', r_ui=4)

Prediction(uid='87', iid='1016', r_ui=4, est=3.6725061814985143, details={'actual_k': 60, 'was_impossible': False})

In [45]:
# 预测评分与实际评分对比
best_knn.predict('174', '368', r_ui=1)

Prediction(uid='174', iid='368', r_ui=1, est=1, details={'actual_k': 27, 'was_impossible': False})

In [46]:
# 经过495次计算，一不小心就用了差不一个小时来训练
len(gs.cv_results['mean_fit_time'])

495